In [ ]:
%load_ext autoreload  
%autoreload 2   

In [ ]:
import os
import openai
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
    set_global_handler
)
from IPython.display import Markdown, display
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import set_global_service_context
from llama_index import ServiceContext, StorageContext
from custom.llms.GeminiLLM import Gemini
from custom.llms.proxy_model import ProxyModel
from llama_index.postprocessor import MetadataReplacementPostProcessor
from llama_index.postprocessor import SentenceTransformerRerank
# llm = Gemini(temperature=0.01, model_name="gemini-pro", max_tokens=2048)
llm = ProxyModel(temperature=0.01, model_name="qwen:7b", max_tokens=2048)
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-zh-v1.5")
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)
set_global_service_context(service_context)
rerank_postprocessor = SentenceTransformerRerank(
            model="BAAI/bge-reranker-large", top_n=10)

In [ ]:
from custom.law_sentence_window import LawSentenceWindowNodeParser
documents = SimpleDirectoryReader("./data/law/").load_data()
node_parser = LawSentenceWindowNodeParser.from_defaults(
            window_size=3,
            window_metadata_key="window",
            original_text_metadata_key="original_text",)
nodes = node_parser.get_nodes_from_documents(documents)
index = VectorStoreIndex(nodes,show_progress=True)

In [ ]:
from llama_index.prompts import ChatPromptTemplate, ChatMessage, MessageRole, PromptTemplate

QA_SYSTEM_PROMPT = "你是一个严谨的中国法律学的专家，你会仔细阅读包含法律条文的资料并给出准确的回答,你的回答都会非常准确"
QA_PROMPT_TMPL_STR = (
    "请你仔细阅读给出的法律法规资料进行回答，如果发现资料无法得到答案，就回答不知道。 \n"
    "仅根据以下搜索出的相关法律法规资料，而不依赖于先验知识，回答问题。\n"
    "每个法律法规资料，都有三个附加信息，1. 该资料所属法律法规；2. 该资料在法律法规中所属的章节；3. 该资料在法律法规中所属的章节中所属的条款。回答法律法规条款内容时，请参考\n"
    "搜索的相关法律法规资料如下所示。\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "问题: {query_str}\n"
    "答案: "
)
REFINE_PROMPT_TMPL_STR = ( 
    "你是一个严谨的中国法律学的专家，你严格按以下方式工作"
    "1.只有原答案为不知道时才进行修正,否则输出原答案的内容。\n"
    "2.如果感到疑惑的时候，就用原答案的内容回答。\n"
    "新的相关法律法规资料: {context_msg}\n"
    "原问题: {query_str}\n"
    "原答案: {existing_answer}\n"
    "新答案: "
)

message_templates = [
            ChatMessage(content=QA_SYSTEM_PROMPT, role=MessageRole.SYSTEM),
            ChatMessage(
                content=QA_PROMPT_TMPL_STR,
                role=MessageRole.USER,
            ),
        ]

query_engine = index.as_query_engine(similarity_top_k=20,node_postprocessors=[
            rerank_postprocessor,
            MetadataReplacementPostProcessor(target_metadata_key="window")
        ])
chat_template = ChatPromptTemplate(message_templates=message_templates)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": chat_template,"response_synthesizer:refine_template": PromptTemplate(REFINE_PROMPT_TMPL_STR)}
)
query_engine.update_prompts("")

In [ ]:

#set_global_handler("simple")
# question="某家超市在促销活动中将产品价格先提高，再打折，实际上消费者并没有得到实际的优惠。违反了消费者权益保护法的哪些条款"
#question="消费者权益保护法中关于商品价格，有哪些条款"
# question="消费者权益保护法第八条的内容"
question="消费者权益保护法第二章的详细内容是什么"
#question="消费者权益保护法第56条的详细内容是什么"
response = query_engine.query(question)
query_engine._get_prompts
print(response)